[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_Code/02_Reacher_Env/02_Default_Environment/03_HF_DIRECT_RAD_REACHER/01_HF_Direct_Reckless_Adaptive_Policy_Training_Reacher.ipynb)

In [ ]:
!pip install gymnasium mujoco stable-baselines3 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# Import the necessary libraries
import gymnasium as gym
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from google.colab import data_table
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Mounted at /content/drive


In [ ]:
# Load the dataframe back from the pickle file
trajectory_df = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/02_trajectories/0_initial_training/0_initial_trajectory_reacher_df.pkl')         # Update directory location 1

In [ ]:
# Display the data frame
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Check the data type of each column
print(type(trajectory_df['Episode'][0]))
print(type(trajectory_df['Timestep'][0]))
print(type(trajectory_df['State'][0]))
print(type(trajectory_df['Action'][0]))
print(type(trajectory_df['Reward'][0]))
print(type(trajectory_df['Next State'][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>


In [ ]:
trajectory_df.dtypes

,0
Episode,int64
Timestep,int64
State,object
Action,object
Reward,float64
Next State,object


HUMAN FEEDBACK IMPLEMENTATION (IDEAL CASE SCENARIO)



In [ ]:
def calculate_different_rewards(trajectory_df):
    """Calculates effort-based reward and returns updated dataframe."""
    df = trajectory_df.copy()

    df['Distance Reward'] = -df['State'].apply(lambda x: np.linalg.norm(np.array(x)[8:10], axis=0))

    df['Smoothness Reward'] = -np.abs(df['State'].apply(lambda x: np.array(x)[6:8]).apply(np.linalg.norm))

    # df['Effort Reward'] = -df['Action'].apply(lambda x: np.linalg.norm(np.array(x)**2, axis=0))
    df['Effort Reward'] = -df['Action'].apply(lambda x: np.linalg.norm(np.array(x), axis=0))
    return df

In [ ]:
different_rewards_df = calculate_different_rewards(trajectory_df)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(different_rewards_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def compute_weighted_rewards(df):
    # Step 1: Create a copy of the dataframe
    df = df.copy()

    # Step 2: Compute min and max values for Smoothness Reward and Effort Reward
    min_smoothness = df["Smoothness Reward"].min()
    max_smoothness = df["Smoothness Reward"].max()
    print("Minimum Smoothness Reward",min_smoothness)
    print("Maximum Smoothness Reward",max_smoothness)


    min_effort = df["Effort Reward"].min()
    max_effort = df["Effort Reward"].max()
    print("Minimum Effort Reward",min_effort)
    print("Maximum Effort Reward",max_effort)

    # Step 3: Compute Smoothness Score Range and Effort Score Range
    xsmooth = 1 / abs(max_smoothness - min_smoothness)
    xeffort = 1 / abs(max_effort - min_effort)

    print(f"Smoothness Score Range: {xsmooth}")
    print(f"Effort Score Range: {xeffort}")

    # Step 4: Compute Smoothness Weight (λ) and Effort Weight (ε) using Softmax
    exp_xsmooth = np.exp(xsmooth)
    exp_xeffort = np.exp(xeffort)

    lambda_smooth = exp_xsmooth / (exp_xsmooth + exp_xeffort)
    epsilon_effort = exp_xeffort / (exp_xsmooth + exp_xeffort)

    # Adjust weights based on bias type
    # bias_type == "conservative":
    lambda_smooth *= 1.6  # Increase smoothness importance                        # bias value 1
    epsilon_effort *= 0.8 # Reduce effort tolerance                              # bias value 2

    print(f"Smoothness Weight (λ): {lambda_smooth}")
    print(f"Effort Weight (ε): {epsilon_effort}")

    # Step 5: Compute Smoothness Weighted Reward (R1) and Effort Weighted Reward (R2)
    df["Smoothness Weighted Reward"] = -df["Distance Reward"] - lambda_smooth * df["Smoothness Reward"]
    df["Effort Weighted Reward"] = -df["Distance Reward"] - epsilon_effort * df["Effort Reward"]

    return df

In [ ]:
reward_df = compute_weighted_rewards(different_rewards_df)

Minimum Smoothness Reward -9.647165745920095
Maximum Smoothness Reward -0.000848030583177548
Minimum Effort Reward -0.5573621988296509
Maximum Effort Reward -0.0008373789023607969
Smoothness Score Range: 0.10366650047303289
Effort Score Range: 1.7968650529021328
Smoothness Weight (λ): 0.2485690597544502
Effort Weight (ε): 0.6757154701227749


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(reward_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Function to recalibrate the rewards
def recalibrate_rewards(df):
    # Create a copy of the dataframe
    df_copy = df.copy()

    alpha = 1
    # Create the 'Recalibrated_rewards' column
    df_copy['Recalibrated Reward'] = df_copy['Reward'] + alpha * (df_copy['Smoothness Weighted Reward'] + df_copy['Effort Weighted Reward'])

    # Get the list of recalibrated rewards
    recalibrated_rewards_list = df_copy['Recalibrated Reward'].tolist()

    return df_copy, recalibrated_rewards_list

In [ ]:
# Apply the function to recalibrate rewards
recalibrated_df, recalibrated_rewards_list = recalibrate_rewards(reward_df)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(recalibrated_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
recalibrated_df.to_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/02_trajectories/1_human_feedback/3_Hf_D_Reckless_Reacher_df_3.pkl')           # Update directory location 2

In [ ]:
# To access the reward for a specific step:
for i, feedback in enumerate(recalibrated_rewards_list):
    human_recalibrated_reward_for_step = recalibrated_rewards_list[i]
    print(f"Recalibrated reward for step {i}: {human_recalibrated_reward_for_step}")

Recalibrated reward for step 0: 0.15798927550372746
Recalibrated reward for step 1: 0.25871164716810147
Recalibrated reward for step 2: 0.36801323383528894
Recalibrated reward for step 3: 0.5221624080360733
Recalibrated reward for step 4: 0.5957984803454983
Recalibrated reward for step 5: 0.5722195618293433
Recalibrated reward for step 6: 0.558810609123115
Recalibrated reward for step 7: 0.5595273120372672
Recalibrated reward for step 8: 0.6570497611950337
Recalibrated reward for step 9: 0.7195042201167005
Recalibrated reward for step 10: 0.839286791092389
Recalibrated reward for step 11: 0.918762161008988
Recalibrated reward for step 12: 0.9696224573530704
Recalibrated reward for step 13: 0.990133373187598
Recalibrated reward for step 14: 1.0338932435803976
Recalibrated reward for step 15: 1.0645409318175203
Recalibrated reward for step 16: 0.9966504898334236
Recalibrated reward for step 17: 1.0062351481747305
Recalibrated reward for step 18: 1.1166100178945106
Recalibrated reward for

In [ ]:
# Step A.5.1: CUSTOM REWARD FUNCTION
def custom_reward(self, env, state, action, next_state, reward, done):
    # Access and recalculate the reward using human_feedback_data or recalibrate_rewards_human function
    global step_counter
    try:
        step_counter
    except NameError:
        step_counter = 0

    reward = recalibrated_rewards_list[step_counter]
    step_counter += 1
    return reward

# Create a new environment class that wraps your original environment and overrides the default reward function with your custom function
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardWrapper, self).__init__(env)

    def step(self, action):
        next_state, reward, terminated, truncated, info = self.env.step(action)
        done = terminated or truncated
        reward = custom_reward(self, self.env, self.last_obs, action, next_state, reward, done)
        # custom_reward should be defined and accessible to your class
        self.last_obs = next_state
        return next_state, reward, terminated, truncated, info

    def reset(self, **kwargs):
        global step_counter
        step_counter = 0
        self.last_obs = self.env.reset(**kwargs)[0]  # Assuming Gymnasium env returns (obs, info)
        return self.last_obs, {}  # Assuming Gymnasium env requires (obs, info)
# Create and wrap the environment with your custom reward wrapper
# env_human = CustomRewardWrapper(gym.make('highway-v0'))

PPO training and Training logs

In [ ]:
drive_log_dir = "/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/00_training_logs/3_log_dir/3_ppo_reacher_hf_direct_reckless_3"          # Update directory location 3

In [ ]:
# Train PPO with Custom Rewards
def train_ppo_with_custom_rewards(log_dir=drive_log_dir, total_timesteps=200000):
    os.makedirs(log_dir, exist_ok=True)
    env = CustomRewardWrapper(gym.make("Reacher-v5"))
    env = Monitor(env, log_dir)
    model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)
    model.learn(total_timesteps=total_timesteps)
    model.save('/content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/01_trained_models/3_ppo_reacher_hf_direct_reckless_3')  # Update directory location 4
    return model, log_dir

In [ ]:
# Execute Training and Convergence Tracking
model, log_dir = train_ppo_with_custom_rewards(total_timesteps=200000)

Using cpu device
Wrapping the env in a DummyVecEnv.
Logging to /content/drive/MyDrive/05_zero_shot_llm_3/03_reacher_data/00_training_logs/3_log_dir/3_ppo_reacher_hf_direct_reckless_3/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | 51.3     |
| time/              |          |
|    fps             | 1193     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 50           |
|    ep_rew_mean          | 51.3         |
| time/                   |              |
|    fps                  | 756          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0032487898 

In [ ]:
log_path = os.path.join(drive_log_dir, "monitor.csv")
df = pd.read_csv(log_path, skiprows=1)
# Ensure episodes are logged correctly
df.reset_index(inplace=True)
df.rename(columns={"index": "episode", "r": "reward", "l": "length", "t": "time_step"}, inplace=True)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(df.head())

,episode,reward,length,time_step
0,0,51.256635,50,6.651103
1,1,51.256635,50,6.692339
2,2,51.256635,50,6.730441
3,3,51.256635,50,6.770488
4,4,51.256635,50,6.812051
